You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Evolution of Artificial Intelligence: Trends, Players, and News

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in today's world
- Brief overview of the latest trends in AI

II. Latest Trends in Artificial Intelligence
- Machine learning and deep learning advancements
- Natural language processing innovations
- AI ethics and regulations updates
- Integration of AI in various industries such as healthcare, finance, and transportation

III. Key Pl

I now can give a great answer

Final Answer: 

# The Evolution of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems. In today's rapidly advancing world, AI plays a crucial role in various sectors, from healthcare to finance to transportation. The latest trends in AI encompass machine learning and deep learning advancements, natural language processing innovations, as well as updates on AI ethics and regulations. As AI continues to evolve, it is essential to stay informed about the latest developments shaping the industry.

## Latest Trends in Artificial Intelligence
Machine learning and deep learning have been at the forefront of AI innovation, enabling machines to learn from data and make decisions. Natural language processing advancements have revolutionized how machines understand and generate human language. Moreover, the ongoing dialogu

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Evolution of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems. In today's rapidly advancing world, AI plays a crucial role in various sectors, from healthcare to finance to transportation. The latest trends in AI encompass machine learning and deep learning advancements, natural language processing innovations, as well as updates on AI ethics and regulations. As AI continues to evolve, it is essential to stay informed about the latest developments shaping the industry.

## Latest Trends in Artificial Intelligence
Machine learning and deep learning have been at the forefront of AI innovation, enabling machines to learn from data and make decisions. Natural language processing advancements have revolutionized how machines understand and generate human language. Moreover, the ongoing dialogue around AI ethics and regulations highlights the importance of responsible AI development and deployment. Across industries such as healthcare, finance, and transportation, AI is being integrated to enhance efficiency and productivity.

## Key Players in the AI Industry
Leading tech companies like Google, Amazon, and Microsoft have been investing heavily in AI research and development, driving innovation in the industry. Meanwhile, emerging startups are making significant contributions to the AI market with groundbreaking solutions. Key research institutions and universities are also pivotal in advancing AI technologies and shaping the future of AI.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in AI technology, such as advancements in computer vision and natural language processing, have been making headlines. The impact of AI on job markets and society is a topic of ongoing discussion, with concerns about automation and job displacement. Ethical considerations in AI development and deployment are crucial for ensuring that AI technologies are used responsibly and ethically.

## Target Audience Analysis
Professionals in the tech industry seeking to stay updated on AI trends, business owners looking to implement AI solutions, and students and researchers interested in the latest advancements in AI are the primary audience for this content. By providing insights into AI trends, key players in the industry, and the latest news in AI, this blog aims to keep readers informed and engaged.

## SEO Keywords
Artificial Intelligence trends, key players in AI industry, latest news in Artificial Intelligence, AI developments 2021

## Resources
- Forbes: Top AI Trends to Watch in 2021
- TechCrunch: The Rise of AI Startups
- Harvard Business Review: Ethical Dilemmas in AI

**Call to Action:** Stay ahead in the world of Artificial Intelligence by subscribing to our newsletter for regular updates on the latest trends, players, and news in AI. 

In conclusion, Artificial Intelligence is a rapidly evolving field with significant implications for various industries. Staying informed about the latest trends, key players, and news in AI is essential for professionals, business owners, and researchers alike. As AI continues to shape the future, it is crucial to be proactive in understanding and adapting to the changes brought about by this transformative technology.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Sucessão patrimonial"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Sucessão patrimonial.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I have all the necessary tools and information to create a comprehensive content plan on the topic of Sucessão patrimonial. By prioritizing the latest trends, key players, and noteworthy news in the field, identifying the target audience and their interests, developing a detailed content outline, and including SEO keywords and relevant data, I can deliver a high-quality and engaging blog article.

Final Answer:

Content Plan Document

Title: Sucessão Patrimonial: A Guide to Managing Your Estate

Outline:

I. Introduction
- Definition of Sucessão patrimonial

I now can give a great answer

Final Answer:
# Sucessão Patrimonial: A Guide to Managing Your Estate

## Introduction

Sucessão patrimonial, also known as estate planning, is the process of arranging and managing the transfer of one's wealth and assets to the next generation or beneficiaries. It involves making important decisions about how your assets will be distributed after your passing, ensuring that your wishes are carried out effectively. Estate planning is crucial for individuals and families to protect their wealth and provide for their loved ones in a tax-efficient manner. Key players in the field include estate planning attorneys, financial advisors specializing in estate planning, and trust companies.

## Latest Trends in Sucessão Patrimonial

In recent years, digital estate planning has become increasingly important as more assets are stored online. It is essential to consider how to manage digital assets such as social media accounts, cryptocurrencies, and online accounts

In [14]:
Markdown(result)

# Sucessão Patrimonial: A Guide to Managing Your Estate

## Introduction

Sucessão patrimonial, also known as estate planning, is the process of arranging and managing the transfer of one's wealth and assets to the next generation or beneficiaries. It involves making important decisions about how your assets will be distributed after your passing, ensuring that your wishes are carried out effectively. Estate planning is crucial for individuals and families to protect their wealth and provide for their loved ones in a tax-efficient manner. Key players in the field include estate planning attorneys, financial advisors specializing in estate planning, and trust companies.

## Latest Trends in Sucessão Patrimonial

In recent years, digital estate planning has become increasingly important as more assets are stored online. It is essential to consider how to manage digital assets such as social media accounts, cryptocurrencies, and online accounts in your estate plan. Additionally, changing tax laws have a significant impact on estate planning strategies, requiring individuals to stay up-to-date on the latest regulations to minimize tax liabilities. Family offices have also emerged as a popular choice for high-net-worth individuals in managing their family wealth, providing comprehensive services for estate planning and wealth management.

## Key Players in Sucessão Patrimonial

Estate planning attorneys play a crucial role in helping individuals and families create comprehensive estate plans that align with their goals and objectives. Financial advisors specializing in estate planning offer valuable insights and advice on investment strategies, insurance planning, and retirement planning to support long-term financial goals. Trust companies and family offices provide professional services for managing family wealth, including investment management, tax planning, and philanthropic giving.

## Noteworthy News in Sucessão Patrimonial

Staying informed about recent court cases related to estate disputes, updates on tax laws affecting estate planning, and success stories of successful estate transitions can provide valuable insights for individuals and families navigating the complexities of estate planning. Learning from real-life examples and understanding the implications of legal and financial changes can help individuals make informed decisions about their estate plans and ensure a smooth transition of wealth to future generations.

## Conclusion

In conclusion, sucessão patrimonial is a critical aspect of financial planning for high-net-worth individuals, families with complex estate planning needs, and business owners planning for succession. By understanding the latest trends and developments in estate planning, working with key players in the field, and staying informed about noteworthy news and updates, individuals can create effective estate plans that protect their assets and provide for their loved ones. Consulting with an estate planning attorney, scheduling a financial planning session, and attending workshops on estate planning best practices are essential steps to ensure a successful estate planning process.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).